In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, ConfusionMatrixDisplay, RocCurveDisplay
import matplotlib.pyplot as plt
import warnings

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

In [2]:
# Import dataset
print("Loading dataset...")
df = pd.read_csv('Walmart_Store_sales.csv')
print("...Done.")
print()

Loading dataset...
...Done.



In [3]:
# Basic stats
print("Number of rows : {}".format(df.shape[0]))
print()

print("Display of df: ")
display(df.head())
print()

print("Basics statistics: ")
data_desc = df.describe(include='all')
display(data_desc)
print()

print("Percentage of missing values: ")
display(round(100*df.isnull().sum()/df.shape[0], 2))

Number of rows : 150

Display of df: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092



Basics statistics: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000



Percentage of missing values: 


Store            0.00
Date            12.00
Weekly_Sales     9.33
Holiday_Flag     8.00
Temperature     12.00
Fuel_Price       9.33
CPI              8.00
Unemployment    10.00
dtype: float64

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         150 non-null    float64
 1   Date          132 non-null    object 
 2   Weekly_Sales  136 non-null    float64
 3   Holiday_Flag  138 non-null    float64
 4   Temperature   132 non-null    float64
 5   Fuel_Price    136 non-null    float64
 6   CPI           138 non-null    float64
 7   Unemployment  135 non-null    float64
dtypes: float64(7), object(1)
memory usage: 9.5+ KB


In [5]:
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')
df['Weekly_Sales'] = df['Weekly_Sales'].round(2)
df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month
df['day'] = df['Date'].dt.day
df['day_of_week'] = df['Date'].dt.day_of_week
df = df.dropna(subset=['Weekly_Sales'])
df = df.sort_values(by=['Store'])

In [6]:
df = df.convert_dtypes()

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 136 entries, 73 to 110
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Store         136 non-null    Int64         
 1   Date          118 non-null    datetime64[ns]
 2   Weekly_Sales  136 non-null    Float64       
 3   Holiday_Flag  125 non-null    Int64         
 4   Temperature   121 non-null    Float64       
 5   Fuel_Price    124 non-null    Float64       
 6   CPI           125 non-null    Float64       
 7   Unemployment  122 non-null    Float64       
 8   year          118 non-null    Int64         
 9   month         118 non-null    Int64         
 10  day           118 non-null    Int64         
 11  day_of_week   118 non-null    Int64         
dtypes: Float64(5), Int64(6), datetime64[ns](1)
memory usage: 15.3 KB


In [8]:
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,year,month,day,day_of_week
73,1,2010-08-27,1449142.92,<NA>,85.22,2.619,211.567306,7.787,2010,8,27,4
44,1,2010-02-12,1641957.44,1,38.51,2.548,211.24217,8.106,2010,2,12,4
42,1,NaT,1661767.33,1,<NA>,3.73,222.439015,6.908,<NA>,<NA>,<NA>,<NA>
74,1,2012-10-19,1508068.77,0,67.97,3.594,223.425723,<NA>,2012,10,19,4
78,1,2011-11-18,1539483.7,0,62.25,3.308,218.220509,7.866,2011,11,18,4


In [9]:
# EDA visualization
fig = px.bar(df, x='Date', y='Weekly_Sales', title='Weekly Sales Over Time', labels={'Date': 'Date', 'Weekly_Sales': 'Weekly Sales'})
# fig.update_xaxes(type='date')
fig.show()

c:\Users\nicol\miniconda3\envs\jedha_env\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  v = v.dt.to_pydatetime()


In [10]:
fig = px.bar(df, x='month', y='Weekly_Sales', title='Weekly Sales / Month', labels={'Date': 'Date', 'Weekly_Sales': 'Weekly Sales'})
fig.show()

In [11]:
fig = px.bar(df, x='day', y='Weekly_Sales', title='Weekly Sales / Day')
fig.show()

In [12]:
fig = px.scatter(df, x='day_of_week', y='Weekly_Sales', title='Weekly Sales / Day of week', labels={'Date': 'Date', 'Weekly_Sales': 'Weekly Sales'})
fig.show()

In [13]:
fig = px.scatter(df, x='Store', y=['Weekly_Sales'], title='Weekly Sales / Store')
fig.show()

In [14]:
fig = px.scatter(data_frame=df, x='Holiday_Flag', y='Weekly_Sales', title='Weekly Sales / Holiday')
fig.show()

In [15]:
fig = px.scatter(df, x='Temperature', y='Weekly_Sales', title='Weekly Sales / Temperature', color='Weekly_Sales')
fig.show()

In [16]:
fig = px.scatter(data_frame=df, x='Fuel_Price', y='Weekly_Sales', title='Weekly Sales / Fuel Price')
fig.show()

In [17]:
# fig = px.scatter(df, x='CPI', y='Weekly_Sales', title='Weekly Sales / CPI')
fig = px.pie(df, values='Weekly_Sales', names='Store', title='Weekly Sales / CPI', hole=.3)
fig.update_traces(sort=False, insidetextorientation='radial', textinfo='value+percent')
fig.show()

In [18]:
fig = px.histogram(df, x='Unemployment', y='Weekly_Sales', title='Weekly Sales Over Time')
fig.show()

In [19]:
# Find the store with the highest weekly sales
max_sales_store = df.groupby('Store')['Weekly_Sales'].sum().idxmax()

# Find the store with the lowest weekly sales
min_sales_store = df.groupby('Store')['Weekly_Sales'].sum().idxmin()

print(f"Store with the highest weekly sales: {max_sales_store}")
print(f"Store with the lowest weekly sales: {min_sales_store}")

Store with the highest weekly sales: 14
Store with the lowest weekly sales: 9


In [20]:
#Enlèvement des Outliers
remove_outlier=['Temperature','Fuel_Price','CPI','Unemployment']

for col in remove_outlier:
    mean = df[col].mean()
    std= df[col].std()

    mask = np.abs((df[col] - mean) <= 3* std) & ((df[col] - mean) >= - 3* std)
    df = df[mask]

display(df.head())
print()
display(df.describe(include='all'))

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,year,month,day,day_of_week
73,1,2010-08-27,1449142.92,<NA>,85.22,2.619,211.567306,7.787,2010,8,27,4
44,1,2010-02-12,1641957.44,1,38.51,2.548,211.24217,8.106,2010,2,12,4
78,1,2011-11-18,1539483.7,0,62.25,3.308,218.220509,7.866,2011,11,18,4
13,1,2012-03-16,1677472.78,0,64.74,3.734,221.211813,7.348,2012,3,16,4
95,1,2010-05-14,1494251.5,0,74.78,2.854,210.337426,7.808,2010,5,14,4


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,year,month,day,day_of_week
count,90.0,80,90.0,80.0,90.0,90.0,90.0,90.0,80.0,80.0,80.0,80.0
mean,9.9,2011-05-17 11:06:00,1233864.509,0.075,61.061,3.318444,179.524905,7.389733,2010.8875,6.3625,16.125,4.0
min,1.0,2010-02-05 00:00:00,268929.03,0.0,18.79,2.548,126.128355,5.143,2010.0,1.0,1.0,4.0
25%,4.0,2010-08-04 06:00:00,561724.0475,0.0,45.3425,2.81475,132.602339,6.64225,2010.0,4.0,10.0,4.0
50%,9.0,2011-05-16 12:00:00,1260826.1,0.0,61.45,3.468,197.166416,7.419,2011.0,6.0,16.5,4.0
75%,15.75,2012-02-18 18:00:00,1807159.02,0.0,75.7925,3.73775,214.855374,8.099,2012.0,8.25,23.25,4.0
max,20.0,2012-10-19 00:00:00,2771397.17,1.0,91.65,4.17,226.968844,9.342,2012.0,12.0,31.0,4.0
std,6.204475,NaN,664725.013301,0.265053,17.74604,0.484399,39.554303,0.982729,0.826672,3.028321,8.521566,0.0


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 90 entries, 73 to 110
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Store         90 non-null     Int64         
 1   Date          80 non-null     datetime64[ns]
 2   Weekly_Sales  90 non-null     Float64       
 3   Holiday_Flag  80 non-null     Int64         
 4   Temperature   90 non-null     Float64       
 5   Fuel_Price    90 non-null     Float64       
 6   CPI           90 non-null     Float64       
 7   Unemployment  90 non-null     Float64       
 8   year          80 non-null     Int64         
 9   month         80 non-null     Int64         
 10  day           80 non-null     Int64         
 11  day_of_week   80 non-null     Int64         
dtypes: Float64(5), Int64(6), datetime64[ns](1)
memory usage: 10.1 KB


In [22]:
# Separate target variable Y from features X
target_name = 'Weekly_Sales'
features_list = ['Store', 'Temperature','Fuel_Price','CPI','Unemployment', 'Holiday_Flag', 'year', 'month', 'day', 'day_of_week']

print("Separating labels from features...")
Y = df.loc[:,target_name]
X = df.loc[:,features_list] # All columns are kept, except the target
print("...Done.")
print(Y.head())
print()
print(X.head())
print()

Separating labels from features...
...Done.
73    1449142.92
44    1641957.44
78     1539483.7
13    1677472.78
95     1494251.5
Name: Weekly_Sales, dtype: Float64

    Store  Temperature  Fuel_Price         CPI  Unemployment  Holiday_Flag  \
73      1        85.22       2.619  211.567306         7.787          <NA>   
44      1        38.51       2.548   211.24217         8.106             1   
78      1        62.25       3.308  218.220509         7.866             0   
13      1        64.74       3.734  221.211813         7.348             0   
95      1        74.78       2.854  210.337426         7.808             0   

    year  month  day  day_of_week  
73  2010      8   27            4  
44  2010      2   12            4  
78  2011     11   18            4  
13  2012      3   16            4  
95  2010      5   14            4  



In [23]:
# First : always divide dataset into train set & test set !!


print("Dividing into train and test sets...")
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
# test_size indicates the proportion of rows from X and Y that will go into the test dataset while 
# maintaining the correspondance between the rows from X and Y 

# random_state is an argument that can be found in all functions that have a pseudo-random behaviour
# if random_state is not stated the function will derive a different random result everytime the cell 
# runs, if random_state is given a value the results will be the same everytime the cell runs while
# each different value of radom_state will derive a specific result
print("...Done.")
print()

Dividing into train and test sets...
...Done.



In [24]:
# *Temperature*, *Fuel_price*, *CPI* and *Unemployment*
# Create pipeline for numeric features
numeric_features = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'year', 'month', 'day', 'day_of_week'] # Names of numeric columns in X_train/X_test
numeric_transformer = Pipeline(
    steps=[
    ('imputer', SimpleImputer(strategy='median')), # missing values will be replaced by columns' median
    ('scaler', StandardScaler())
])

# Create pipeline for categorical features
categorical_features = ['Store', 'Holiday_Flag'] # Names of categorical columns in X_train/X_test
categorical_transformer = Pipeline(
    steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # missing values will be replaced by most frequent value
    ('encoder', OneHotEncoder(drop='first')) # first column will be dropped to avoid creating correlations between features
    ])

In [25]:
# Use ColumnTransformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [26]:
# Preprocessings on train set
print("Performing preprocessings on train set...")
print(X_train.head())
X_train = preprocessor.fit_transform(X_train)
print('...Done.')
print(X_train[0:5]) # MUST use this syntax because X_train is a numpy array and not a pandas DataFrame anymore
print()

# Preprocessings on test set
print("Performing preprocessings on test set...")
print(X_test.head()) 
X_test = preprocessor.transform(X_test) # Don't fit again !! The test set is used for validating decisions
# we made based on the training set, therefore we can only apply transformations that were parametered using the training set.
# Otherwise this creates what is called a leak from the test set which will introduce a bias in all your results.
print('...Done.')
print(X_test[0:5,:]) # MUST use this syntax because X_test is a numpy array and not a pandas DataFrame anymore
print()

Performing preprocessings on train set...
     Store  Temperature  Fuel_Price         CPI  Unemployment  Holiday_Flag  \
17      18        21.33       2.788  131.527903         9.202             0   
107      8        33.34       2.548  214.621419         6.299             1   
136      4        84.59       3.469    129.1125         5.644          <NA>   
45       2        54.63       3.555  220.275944         7.057             0   
59      14        36.85       3.695  189.842483         8.424             0   

     year  month   day  day_of_week  
17   <NA>   <NA>  <NA>         <NA>  
107  2010      2    12            4  
136  2011      7     8            4  
45   2012      2    24            4  
59   2012      2    17            4  
...Done.
[[-2.27582047 -1.16285602 -1.11596452  1.7922351   0.0696733  -0.05386943
   0.04276567  0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0

In [27]:
# Train model
print("Train model...")
regressor = LinearRegression()
regressor.fit(X_train, Y_train)
print("...Done.")

Train model...
...Done.


In [28]:
# Predictions on training set
print("Predictions on training set...")
Y_train_pred = regressor.predict(X_train)
print("...Done.")
print(Y_train_pred)
print()

Predictions on training set...
...Done.
[1194094.37909033  941688.01801649 2216618.73914576 1872207.59829593
 2010659.65986005 1895038.91381621 1331299.01720209 1856834.00023281
  975051.90833186 1148775.12960324 1435389.38445369 1558742.60080193
 1930116.4439453  1113764.84150686  434264.94737672  403621.56384615
 1834427.0865165  1685084.16548292  962897.6796131  1975100.78084853
 1990901.11864515 1556816.70583736  411845.67750437  366562.17083705
  889821.23198351  486924.38767411 1551987.91310997 1318619.5242274
  461790.44199686 1549420.52115507  482585.08787275  261940.92963543
  441006.22561743  471496.99054549 2332954.4730741  1641766.28451708
  493510.20941151 1605576.61927162 1912770.87619465 1935053.69578529
  738354.34666017 1670240.64909567 1614927.90149773 1128776.00185463
  379585.8447178  1337236.29707241 2122180.97918884 1561405.22923066
 1867108.22170177 1985399.52415744 2074725.63292462  470524.41490035
 2093308.12433729 1496263.74450793  876815.87878407  161223.4729

In [29]:
# Predictions on test set
print("Predictions on test set...")
Y_test_pred = regressor.predict(X_test)
print("...Done.")
print(Y_test_pred)
print()

Predictions on test set...
...Done.
[1653273.2067103   460792.35103769 2102650.93264067  763635.84216257
  553377.51683069 1691201.17698516  529049.26127522  816663.69357094
  972323.39586285 2141772.46038967 2008103.32993288  448712.3448191
  706684.33612837 1980678.21267392 1938460.26043503  458833.16014607
  211453.27754793 2066948.43763656]



In [30]:
# Print R^2 scores
print("R2 score on training set : ", r2_score(Y_train, Y_train_pred))
print("R2 score on test set : ", r2_score(Y_test, Y_test_pred))

R2 score on training set :  0.9851496102618137
R2 score on test set :  0.9485111549400768


In [31]:
regressor.coef_

array([-3.61663975e+04, -7.15683859e+04,  6.82732243e+04, -4.42398557e+04,
        4.53200364e+04,  3.92617752e+04, -4.33381699e+04,  1.28056854e-09,
        3.12886044e+05, -1.22635256e+06,  6.95825007e+05, -1.31627293e+06,
        9.21586900e+04, -1.01520111e+06, -8.24558998e+05, -1.17615752e+06,
        4.05006177e+05,  1.44042149e+04,  4.76998253e+05,  5.10505086e+05,
       -7.55650645e+05, -1.06509632e+06, -7.52244620e+05, -3.41944380e+05,
       -1.08543443e+04,  4.78084148e+05,  4.00399197e+04])

In [32]:
column_names = []
for name, pipeline, features_list in preprocessor.transformers_: # loop over pipelines
    if name == 'num': # if pipeline is for numeric variables
        features = features_list # just get the names of columns to which it has been applied
    else: # if pipeline is for categorical variables
        features = pipeline.named_steps['encoder'].get_feature_names_out() # get output columns names from OneHotEncoder
    column_names.extend(features) # concatenate features names
        
print("Names of columns corresponding to each coefficient: ", column_names)

Names of columns corresponding to each coefficient:  ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'year', 'month', 'day', 'day_of_week', 'x0_2.0', 'x0_3.0', 'x0_4.0', 'x0_5.0', 'x0_6.0', 'x0_7.0', 'x0_8.0', 'x0_9.0', 'x0_10.0', 'x0_11.0', 'x0_13.0', 'x0_14.0', 'x0_15.0', 'x0_16.0', 'x0_17.0', 'x0_18.0', 'x0_19.0', 'x0_20.0', 'x1_1.0']


In [33]:
# Create a pandas DataFrame
coefs = pd.DataFrame(index = column_names, data = regressor.coef_.transpose(), columns=["coefficients"])
mask = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'year', 'month', 'day', 'day_of_week']
coefs = coefs.loc[mask]

In [34]:
# Compute abs() and sort values
feature_importance = abs(coefs).sort_values(by = 'coefficients')
feature_importance

,coefficients
day_of_week,1.280569e-09
Temperature,3.616640e+04
month,3.926178e+04
day,4.333817e+04
Unemployment,4.423986e+04
year,4.532004e+04
CPI,6.827322e+04
Fuel_Price,7.156839e+04


In [35]:
# Plot coefficients
fig = px.bar(feature_importance, orientation = 'h')
fig.update_layout(showlegend = False, 
                  margin = {'l': 120} # to avoid cropping of column names
                 )
fig.show()